# 0. Установка и Импорт

In [1]:
!pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 24.8 MB/s eta 0:00:00


In [2]:
!pip install vosk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 14.6 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=351d789420d9616f6d715daf840903e5dae1b0ca21151ba91bd3b91494e9e632
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd19e6803d7eb919b563dd11de380e4304e332
Successfully built srt


In [3]:
!apt update
!apt install ffmpeg
!mkdir -p ~/vosk-models && cd ~/vosk-models
!wget https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip
!unzip vosk-model-small-ru-0.22.zip
!rm vosk-model-small-ru-0.22.zip

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,438 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelea

In [2]:
import speech_recognition as sr
from pydub import AudioSegment
from vosk import Model, KaldiRecognizer
import wave
import json
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from google.colab import drive
import pandas as pd

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = "your_path"
name_lecture = "Формальные языки и трансляции 5. Построение минимального автомата. Праволинейные грамматики."
audio_path_wav = path + name_lecture + "wav"
audio_path_mp3 = path + name_lecture + "mp3"

name_test = "audio_test."
test_path_wav = path + name_test + "wav"
test_path_mp3 = path + name_test + "mp3"

# 1. Пишем функции для транскрибации аудио для различных моделей

## 1.1 SpeechRecognition

In [7]:
def transcribe_audio_speechRecognition(wav_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='ru-RU')
            return text
        except sr.UnknownValueError:
            return "Не удалось распознать аудио"
        except sr.RequestError as e:
            return f"Ошибка сервиса: {e}"


speechRecognition_text = transcribe_audio_speechRecognition(audio_path_wav)
speechRecognition_test = transcribe_audio_speechRecognition(test_path_wav)

In [8]:
print(speechRecognition_text)
print(speechRecognition_test)

Ошибка сервиса: recognition connection failed: [Errno 32] Broken pipe
1 2 3 тестирование


## 1.2 Whisper

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
whisper_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    "openai/whisper-large-v3", torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
whisper_model.to(device)
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
pipe = pipeline(
    "automatic-speech-recognition",
    model=whisper_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [12]:
whisper_text = pipe(audio_path_wav)
whisper_test = pipe(test_path_wav)
print(whisper_text["text"])
print(whisper_test["text"])

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 так всем доброго дня мы сегодня с вами продолжаем наш курс и наша цель на сегодня закончить рассмотрение большой темы посвященные конечным автоматом а и перед началом нашего занятия пока мы тут настраивали я выписал некоторые факты которые нам сегодня потребуется доказать давайте вспомним что у нас было в предыдущей серии ничего не было да да мы рассмотрели с вами с теоретической точки зрения как устроена минимизация и мы с вами поняли что минимальный pdk существует единственной источностью до изоморфизма единственный вопрос который у нас остался как этот автомат строить точнее алгоритм по которым мы можем строить минимальный автомат а и для этого мы ввели несколько определений сейчас я их повторю на доске точнее на презентации мы с вами доказали существование снасть автомата и вот наша цель сегодня будет вот такая то есть это во первых построить алгоритм нахождения минимального пдк дальше ввести понятие новые новые понятия в нашем курсе который будет называться порождающей грамматики

In [ ]:
del whisper_model

## 1.3 Wav2vec

In [ ]:
#!pip install huggingsound
!pip install transformers --upgrade
!pip install torch -U

In [3]:
from huggingsound import SpeechRecognitionModel

model_wav2vec = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-russian", device="cuda")

INFO:huggingsound.speech_recognition.model:Loading model...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [6]:
wav2vec_test = model_wav2vec.transcribe([test_path_mp3])

100%|██████████| 1/1 [00:04<00:00,  4.41s/it]


In [8]:
print(wav2vec_test[0]['transcription'])

один два три тестирования


In [9]:
wav2vec_text = model_wav2vec.transcribe([audio_path_mp3])

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:370: UserWarning: cuDNN cannot be used for large non-batch-splittable convolutions if the V8 API is not enabled or before cuDNN version 9.3+. Consider upgrading cuDNN and/or enabling the V8 API for better efficiency. (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:430.)
  return F.conv1d(
  0%|          | 0/1 [02:17<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.41 GiB. GPU 0 has a total capacity of 14.75 GiB of which 12.90 GiB is free. Process 62625 has 1.85 GiB memory in use. Of the allocated memory 1.70 GiB is allocated by PyTorch, and 26.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(wav2vec_text[0]['transcription'])

## 1.4 Vosk

In [10]:
model_path = "/content/vosk-model-small-ru-0.22"
model_vosk = Model(model_path)
recognizer = KaldiRecognizer(model_vosk, 16000)  # 16000 - частота дискретизации аудиофайла

In [11]:
def transcribe_audio_vosk(audio_path):
    with wave.open(audio_path, "rb") as wf:
        if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
            return "Аудиофайл должен быть в моно, с частотой дискретизации 16000 и 16-битным форматом."

        result_text = ""
        while True:
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            if recognizer.AcceptWaveform(data):
                result = json.loads(recognizer.Result())
                result_text += result.get("text", "") + " "
        result = json.loads(recognizer.FinalResult())
        result_text += result.get("text", "")
        return result_text

In [13]:
vosk_test = transcribe_audio_vosk(test_path_wav)

In [16]:
vosk_test

один два три тестирование


In [17]:
vosk_text = transcribe_audio_vosk(audio_path_wav)